In [34]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [47]:
import cv2
import os
import numpy as np
import tensorflow as tf
from pathlib import Path
from collections import deque, Counter


# -------------------------------
# CONFIG
# -------------------------------
NUM_IMAGES = 300
FACES_DIR = Path("data/faces")
EMB_DIR = Path("data/embeddings")
EMB_DIR.mkdir(exist_ok=True)

MODEL_PATH = r'/home/sandeshprasai/Final_Semester_Project/AI_Attendance_System/ai-ml-model/src/models/best_face_model.keras'

# -------------------------------
# LOAD MODEL
# -------------------------------
model = tf.keras.models.load_model(MODEL_PATH)

# -------------------------------
# FACE EXTRACTION (for enrollment)
# -------------------------------
def extract_face(img):
    # For controlled enrollment, just resize the whole image
    return cv2.resize(img, (112, 112))

def get_embedding(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    face = face.astype("float32") / 255.0
    face = (face - 0.5) / 0.5
    face = np.expand_dims(face, axis=0)
    emb = model.predict(face, verbose=0)
    return emb[0]

# -------------------------------
# STEP 1: CAPTURE IMAGES
# -------------------------------
person_name = input("Enter person name: ").strip()
person_dir = FACES_DIR / person_name
person_dir.mkdir(parents=True, exist_ok=True)

cap = cv2.VideoCapture("http://192.168.1.12:4747/video")
count = 0
print("Press 'c' to capture | 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # rotate if camera is rotated
    frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)

    cv2.imshow("Capture Face", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('c'):
        path = person_dir / f"img_{count}.jpg"
        cv2.imwrite(str(path), frame)
        print(f"Saved {path}")
        count += 1

    if key == ord('q') or count >= NUM_IMAGES:
        break

cap.release()
cv2.destroyAllWindows()

# -------------------------------
# STEP 2: GENERATE EMBEDDINGS
# -------------------------------
embeddings = []
for img_path in person_dir.glob("*.jpg"):
    img = cv2.imread(str(img_path))
    face = extract_face(img)
    if face is None:
        print(f"Failed to extract face from {img_path}")
        continue
    embeddings.append(get_embedding(face))

if len(embeddings) > 0:
    reference_embedding = np.mean(np.array(embeddings), axis=0)
    emb_path = EMB_DIR / f"{person_name}.npy"
    np.save(emb_path, reference_embedding)
    print(f"Saved embedding: {emb_path}")
else:
    print("No embeddings generated. Check captured images.")

# -------------------------------
# STEP 3: LOAD ALL EMBEDDINGS
# -------------------------------
known_embeddings = {}
for emb_file in EMB_DIR.glob("*.npy"):
    name = emb_file.stem
    known_embeddings[name] = np.load(emb_file)
print(f"Loaded embeddings for: {list(known_embeddings.keys())}")

# -------------------------------
# STEP 4: LIVE FACE RECOGNITION
# -------------------------------
recent_predictions = deque(maxlen=5)
SIMILARITY_THRESHOLD = 0.8  # increase threshold for more confident recognition

cap = cv2.VideoCapture("http://192.168.1.12:4747/video")
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)

    face = extract_face(frame)
    if face is not None:
        emb = get_embedding(face)
        name, score = recognize_face(emb, known_embeddings, threshold=SIMILARITY_THRESHOLD)
        recent_predictions.append(name)

        # Majority vote for steady output
        name_smoothed = Counter(recent_predictions).most_common(1)[0][0]

        label = f"{name_smoothed} ({score:.2f})"
        color = (0, 255, 0) if name_smoothed != "Unknown" else (0, 0, 255)
        cv2.putText(frame, label, (30, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    cv2.imshow("Face Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Press 'c' to capture | 'q' to quit
No embeddings generated. Check captured images.
Loaded embeddings for: ['saurab', 'sandesh', 'sandesh_glasses']


[tcp @ 0x234fdc40] Connection to tcp://192.168.1.12:4747 failed: Connection refused
[tcp @ 0x2363d080] Connection to tcp://192.168.1.12:4747 failed: Connection refused


In [52]:
recent_predictions = deque(maxlen=5)
SIMILARITY_THRESHOLD = 0.8  # increase threshold for more confident recognition

cap = cv2.VideoCapture("http://192.168.1.12:4747/video")
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)

    face = extract_face(frame)
    if face is not None:
        emb = get_embedding(face)
        name, score = recognize_face(emb, known_embeddings, threshold=SIMILARITY_THRESHOLD)
        recent_predictions.append(name)

        # Majority vote for steady output
        name_smoothed = Counter(recent_predictions).most_common(1)[0][0]

        label = f"{name_smoothed} ({score:.2f})"
        color = (0, 255, 0) if name_smoothed != "Unknown" else (0, 0, 255)
        cv2.putText(frame, label, (30, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    cv2.imshow("Face Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()